## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-26-06-52-49 +0000,nypost,Arkansas man who disappeared with his ailing d...,https://nypost.com/2026/01/26/us-news/arkansas...
1,2026-01-26-06-49-21 +0000,bbc,Caribbean cannabis growers eye budding domesti...,https://www.bbc.com/news/articles/c62ndp17jv0o...
2,2026-01-26-06-47-20 +0000,nyt,The Woman Who Stands Between Donald Trump and ...,https://www.nytimes.com/2026/01/25/world/europ...
3,2026-01-26-06-41-00 +0000,wsj,Ryanair Upgrades Fiscal 2026 Outlook on Strong...,https://www.wsj.com/business/earnings/ryanair-...
4,2026-01-26-06-39-07 +0000,bbc,Japanese ex-soldier reaches settlements in lan...,https://www.bbc.com/news/articles/cr4k2kw500zo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2497/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
23,trump,31
48,shooting,27
40,minneapolis,22
78,ice,14
88,minnesota,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
48,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...,132
186,2026-01-25-10-12-37 +0000,nypost,Senate Dems could force gov’t shutdown over $6...,https://nypost.com/2026/01/25/us-news/senate-d...,91
86,2026-01-25-22-48-00 +0000,wsj,Senate Democrats angered by the deadly shootin...,https://www.wsj.com/politics/policy/senate-dem...,89
10,2026-01-26-05-57-05 +0000,bbc,Minneapolis protesters march against ICE as go...,https://www.bbc.com/news/articles/cr571qg4m61o...,87
130,2026-01-25-19-09-57 +0000,latimes,Trump's playbook falters in crisis response to...,https://www.latimes.com/politics/story/2026-01...,86


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
48,132,2026-01-26-01-57-16 +0000,nypost,Trump ‘reviewing everything’ about fatal shoot...,https://nypost.com/2026/01/25/us-news/trump-re...
29,48,2026-01-26-04-30-19 +0000,nypost,Violent anti-ICE protesters swarm Minnesota ho...,https://nypost.com/2026/01/25/us-news/anti-ice...
27,44,2026-01-26-04-38-19 +0000,bbc,"At least seven dead and 800,000 without power ...",https://www.bbc.com/news/articles/c1klrmz90yjo...
86,35,2026-01-25-22-48-00 +0000,wsj,Senate Democrats angered by the deadly shootin...,https://www.wsj.com/politics/policy/senate-dem...
151,31,2026-01-25-15-41-03 +0000,cbc,"No plans for China free trade deal, Carney say...",https://www.cbc.ca/news/politics/carney-canada...
177,29,2026-01-25-11-00-39 +0000,nypost,NYC homeless living high life under Mayor Mamd...,https://nypost.com/2026/01/25/us-news/nyc-home...
55,26,2026-01-26-01-21-56 +0000,nypost,Daughter of top Iranian official ousted from E...,https://nypost.com/2026/01/25/us-news/emory-un...
140,26,2026-01-25-17-00-00 +0000,wsj,U.S. Speaker of the House Mike Johnson has hel...,https://www.wsj.com/politics/republicans-house...
44,24,2026-01-26-02-09-44 +0000,startribune,The latest: Mounting calls for DHS surge to en...,https://www.startribune.com/ice-raids-minnesot...
83,23,2026-01-25-23-09-32 +0000,wapo,ICU nurse fatally shot by Border Patrol in Min...,https://www.washingtonpost.com/nation/2026/01/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
